In [6]:
#this code just writes to the file not important otherwise


from pandas._libs.tslibs import dtypes
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import norm
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.ensemble import RandomForestClassifier
encoder = LabelEncoder()
# load training data
train = pd.read_csv('/content/DataQuest Dataset - train_data.csv')
test = pd.read_csv('/content/DataQuest Dataset - test_data.csv')
finalTest = pd.read_csv('/content/DataQuest Dataset - test_data.csv')

encoded = encoder.fit_transform(train['MealPlan'])
encoded1 = encoder.fit_transform(test['MealPlan'])
encoded2 = encoder.fit_transform(train['RoomType'])
encoded3 = encoder.fit_transform(test['RoomType'])
encoded4 = encoder.fit_transform(train['MarketSegment'])
encoded5 = encoder.fit_transform(test['MarketSegment'])
encoded6 = encoder.fit_transform(train['BookingStatus'])
encoded7 = encoder.fit_transform(test['BookingStatus'])

# replace the original 'Meal Plan' column with the encoded values
train['MealPlan'] = encoded
test['MealPlan'] = encoded1
train['RoomType'] = encoded2
test['RoomType'] = encoded3
train['MarketSegment'] = encoded4
test['MarketSegment'] = encoded5
train['BookingStatus'] = encoded6
test['BookingStatus'] = encoded7


from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
train['CancelRatio'] = train['NumPrevCancellations'] / (train['NumPreviousNonCancelled'] + train['NumPrevCancellations'])
train['CancelRatio'] = train['CancelRatio'].replace(np.nan, 0)
train['NumGuests'] = train['NumAdults'] + train['NumChildren']
train['Nights'] = train['NumWeekendNights'] + train['NumWeekNights']
train['Price'] = train['Nights'] * train['AvgRoomPrice']

train.drop('Parking',axis=1)

train['Date'] = pd.to_datetime(train['ArrivalYear'].astype(str) + '-' + train['ArrivalMonth'].astype(str).str.zfill(2) + '-' + train['ArrivalDate'].astype(str).str.zfill(2), errors='coerce')
cat_features = ['MealPlan', 'Parking', 'RoomType', 'MarketSegment', 'RepeatedGuest']


# create the transformer for one-hot encoding
# calculate number of days since first date
train = train.sort_values(by='Date')
first_date = train['Date'].iloc[0]
train['DaysSinceFirst'] = (train['Date'] - first_date).dt.days

test['CancelRatio'] = test['NumPrevCancellations'] / (test['NumPreviousNonCancelled'] + test['NumPrevCancellations'])
test['CancelRatio'] = test['CancelRatio'].replace(np.nan, 0)
test['NumGuests'] = test['NumAdults'] + test['NumChildren']
test['Nights'] = test['NumWeekendNights'] + test['NumWeekNights']
test['Price'] = test['Nights'] * test['AvgRoomPrice']

test.drop('Parking',axis=1)

test['Date'] = pd.to_datetime(test['ArrivalYear'].astype(str) + '-' + test['ArrivalMonth'].astype(str).str.zfill(2) + '-' + test['ArrivalDate'].astype(str).str.zfill(2), errors='coerce')
cat_features = ['MealPlan', 'Parking', 'RoomType', 'MarketSegment', 'RepeatedGuest']


# create the transformer for one-hot encoding
# calculate number of days since first date
test = test.sort_values(by='Date')
first_date = test['Date'].iloc[0]
test['DaysSinceFirst'] = (test['Date'] - first_date).dt.days

train.replace([np.inf, -np.inf], np.nan, inplace=True)
train.replace([np.inf, -np.inf], np.nan, inplace=True)
train=train.dropna()

test.replace([np.inf, -np.inf], np.nan, inplace=True)
test.replace([np.inf, -np.inf], np.nan, inplace=True)
test=test.dropna()
x = train.loc[:, (train.columns != 'BookingStatus') & (train.columns != 'BookingID') & (train.columns != 'Date')]
y = train.BookingStatus # Target variable
# preprocess training data
# ...
X_train = train.loc[:, (train.columns != 'BookingStatus') & (train.columns != 'BookingID') & (train.columns != 'Date')]
y_train = train['BookingStatus']
# train random forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
#dtypes(X_train)
rf.fit(X_train, y_train)

# load test data
X_test = test.loc[:, (test.columns != 'BookingStatus') & (test.columns != 'BookingID') & (test.columns != 'Date')]
y_test = test['BookingStatus']

# preprocess test data
# ...


missing_data = test
missing_data = test.loc[:, (test.columns != 'BookingStatus') & (test.columns != 'BookingID') & (test.columns != 'Date')]
predicted_status = rf.predict(missing_data)
test['BookingStatus'] = predicted_status


# evaluate model's performance
test['BookingStatus'] = test['BookingStatus'].map({0: 'Canceled', 1: "Not_Canceled"})
test = test.sort_values(by='BookingID')
important = test['BookingStatus']
finalTest['BookingStatus'] = important
finalTest.to_csv('PredictedData.csv')


